In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import math
from scipy import interpolate, stats
import joblib

In [3]:
df = pd.read_csv("../joined_combined_filtered_mined_soil_water_df_td_17_4.csv")

/home/draget/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (17,19,23,39,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
fdf = df[(df["pool"] != True) & (df["secondfloor"] < 1) & (df["mhome"] < 1)  & (df["pers_prop_val"] < 1) & (df["parval"] < 1000000) & (df["parval"] > 10000) & (df["lot_area"] < 500)].copy()
fdf["add_g_size"] = fdf["addsize"] + fdf["gize"]
feature_df = fdf[["lot_area", "TotalWater", "water_dist", "castorieindex", "firstfloor", "city_dist", "parval"]]
feature_df.shape

(15426, 7)

In [5]:
feature_df = feature_df.dropna()
n = (feature_df.shape[1] - 1)
X = feature_df[feature_df.columns[0:n]].values
y = feature_df[feature_df.columns[n]].values
feature_df.shape

(14439, 7)

In [6]:
n_split = int(0.85*len(X))
X,y = shuffle(X,y)
X_train = X[0:n_split]
y_train = y[0:n_split]
X_test = X[n_split:]
y_test = y[n_split:]

In [7]:
regr = RandomForestRegressor(n_estimators = 100, min_samples_split = 50, min_samples_leaf = 20)

In [8]:
regr.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=20,
                      min_samples_split=50, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [9]:
regr.score(X_test, y_test)

0.5317921647956995

In [10]:
regr.feature_importances_

array([0.60510705, 0.09430442, 0.02863637, 0.0185514 , 0.16421824,
       0.08918251])

In [11]:
y_pred = regr.predict(X_test)

In [12]:
test = [regr.predict([[6000,w,11,70,0,0,30]])[0] for w in np.linspace(2.5,5.5,20)]

ValueError: Number of features of the model must match the input. Model n_features is 6 and input n_features is 7 

In [ ]:
test = [regr.predict([[a,5,11,60,0,20]])[0] for a in np.linspace(3,100,20)]

In [ ]:
test = [regr.predict([[100,4.5,x,60,0,30]])[0] for x in np.linspace(3,40,20)]

In [ ]:
plt.scatter(np.linspace(3,100,20), test)

In [ ]:
regr.predict([[2000,4,11,70,800,0,30]])

In [ ]:
plt.scatter(y_pred, y_test)

In [94]:
math.sqrt(mean_squared_error(y_test, y_pred))

139143.4989190485

In [ ]:
feature_df.to_csv("fdf.csv")

In [ ]:
joblib.dump(regr, 'regr.pkl') 


In [ ]:
df["lot_area"].value_counts(bins = 100, sort = False)

In [13]:
features = ["lot_area", "TotalWater", "water_dist", "castorieindex", "firstfloor", "city_dist"]

In [90]:
args = [20,3,11,95,0,35]

In [91]:
filter_series = pd.Series(dtype = bool, data = [True]*len(fdf))

for i, f in enumerate(features):
    iqr = fdf[f].quantile(0.75) - fdf[f].quantile(0.25)
    print(iqr*.10)
    filter_series = filter_series & (fdf[f] <= (args[i] + iqr*0.10)) & (fdf[f] >= (args[i] - iqr*0.10))

3.6210000000000004
0.16770370370370302
0.9143764173819093
4.1000000000000005
128.8
2.199501153258344


In [93]:
fdf[filter_series]["parval"].quantile(0.9)

/home/draget/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


294187.0

In [89]:
fdf[features]

,lot_area,TotalWater,water_dist,castorieindex,firstfloor,city_dist
0,20.88,2.926667,11.698670,95.0,0.0,34.860588
1,8.86,4.360258,8.498322,45.0,1148.0,32.697260
3,19.86,3.223960,1.441799,72.0,0.0,18.230165
4,6.49,4.061667,9.344364,NaN,1920.0,71.870104
5,2.07,3.600193,11.246829,NaN,1935.0,74.352494
...,...,...,...,...,...,...
25331,54.36,4.603704,19.290770,95.0,0.0,10.462060
25332,40.00,4.603704,12.898458,100.0,1158.0,10.482185
25337,35.08,4.043617,20.237908,100.0,0.0,11.607682
25339,19.00,4.603704,15.507714,95.0,0.0,8.486162
